## Utilities

In [1]:
#!fsharp
module Utilities =
    let rec combinations accumulator desiredListSize possibleListItems = seq {
        match desiredListSize, possibleListItems with
        | currentListSize, firstListElement::remainderOfList ->
            if currentListSize > 0 then yield! combinations (firstListElement::accumulator) (currentListSize - 1) remainderOfList
            if currentListSize >= 0 then yield! combinations accumulator currentListSize remainderOfList
        | 0, [] -> yield accumulator
        | _, [] -> ()
    }

## Day 1
Before you leave, the Elves in accounting just need you to fix your expense report (your puzzle input); apparently, something isn't quite adding up.

Specifically, they need you to find the two entries that sum to 2020 and then multiply those two numbers together.

For example, suppose your expense report contained the following:

1721
979
366
299
675
1456
In this list, the two entries that sum to 2020 are 1721 and 299. Multiplying them together produces 1721 * 299 = 514579, so the correct answer is 514579.

Of course, your expense report is much larger. Find the two entries that sum to 2020; what do you get if you multiply them together?

### Shared Setup Code

In [1]:
#!fsharp
let input = 
    [
        1939; 1585; 1712; 1600; 1370; 1447; 1247; 1446; 1323; 1713; 1277; 1946; 1677; 1428; 1231; 1481; 1976; 1709; 1508; 1668; 1302; 77; 
        1351; 1605; 1999; 1982; 1583; 1756; 1957; 1624; 1745; 1938; 1784; 1403; 1642; 1691; 569; 1762; 1555; 1937; 1383; 1897; 1334; 1965;
        1683; 1475; 1776; 1791; 1707; 1987; 1233; 1416; 1769; 1345; 1874; 1255; 1744; 1944; 1404; 1360; 1304; 1417; 1977; 1656; 790; 1788;
        1353; 1296; 1673; 1810; 1684; 1742; 1425; 1887; 1444; 1352; 1229; 1414; 1493; 1402; 1947; 1669; 1412; 1531; 1474; 1637; 1314; 1607;
        1829; 1923; 1949; 1757; 1307; 1714; 1748; 1550; 1372; 1615; 1235; 1272; 1408; 1749; 1687; 1613; 1528; 1561; 341; 1308; 1660; 1667;
        1313; 1991; 1675; 1394; 1704; 1303; 1440; 1592; 1857; 1752; 1839; 1397; 1699; 1426; 1878; 1759; 1814; 1096; 372; 1596; 1500; 1774;
        1627; 1696; 1851; 1020; 1819; 1292; 1616; 1672; 1279; 1543; 1526; 1682; 1568; 1582; 1921; 922; 1773; 1482; 1238; 1973; 1517; 1909;
        409; 1634; 1468; 1445; 1801; 1631; 1407; 1820; 1603; 1495; 1333; 1241; 1849; 82; 1339; 1413; 90; 1662; 1291; 1740; 1340; 1365; 2003;
        1546; 1621; 1650; 1518; 1807; 1382; 1433; 1968; 1940; 1986; 1437; 1651; 1237; 1862; 1409; 1200; 2002; 2009; 1735; 1487; 1706; 1643; 1505; 
    ]

### Part 1

In [1]:
#!fsharp
seq {
    for x in input do 
        let y = 2020 - x
        if input |> List.contains y then yield sprintf $"Answer: x: {x} y: {y}, x * y : {x * y}"
}
|> Seq.head 

Answer: x: 1938 y: 82, x * y : 158916

### Part 2

In [1]:
#!fsharp
seq {
    for x in input do
        for y in input do 
            let z = 2020 - (x + y)
            if input |> List.contains z then yield $"Answer: x: {x} y: {y}, z: {z}, x * y * z : {x * y * z}"
}
|> Seq.head

Answer: x: 1307 y: 341, z: 372, x * y * z : 165795564

## Day 2

Your flight departs in a few days from the coastal airport; the easiest way down to the coast from here is via toboggan.

The shopkeeper at the North Pole Toboggan Rental Shop is having a bad day. "Something's wrong with our computers; we can't log in!" You ask if you can take a look.

Their password database seems to be a little corrupted: some of the passwords wouldn't have been allowed by the Official Toboggan Corporate Policy that was in effect when they were chosen.

To try to debug the problem, they have created a list (your puzzle input) of passwords (according to the corrupted database) and the corporate policy when that password was set.

For example, suppose you have the following list:

1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
Each line gives the password policy and then the password. The password policy indicates the lowest and highest number of times a given letter must appear for the password to be valid. For example, 1-3 a means that the password must contain a at least 1 time and at most 3 times.

In the above example, 2 passwords are valid. The middle password, cdefg, is not; it contains no instances of b, but needs at least 1. The first and third passwords are valid: they contain one a or nine c, both within the limits of their respective policies.

How many passwords are valid according to their policies?

### Input

In [1]:
#!fsharp
open System.Text.RegularExpressions

type PasswordRow =
    {
        LowerLimit: int
        UpperLimit: int
        TargetCharacter: char
        AttemptedPassword: char []
    }

let day2Input = File.ReadAllText("./Day2Input.txt")
let pattern = @"(?<lower>\d+)-(?<upper>\d+) (?<target>\w){1}: (?<password>\w+)"
let rows =
    Regex.Matches(day2Input, pattern, RegexOptions.Multiline)
    |> Seq.cast<Match>
    |> Seq.map (fun m -> 
        {
            LowerLimit = m.Groups.["lower"].Value |> int
            UpperLimit = m.Groups.["upper"].Value |> int
            TargetCharacter = m.Groups.["target"].Value |> char
            AttemptedPassword = m.Groups.["password"].Value.ToCharArray()
        })

### Part 1

In [1]:
#!fsharp
let validPassword passwordRow =
    let targetCharCount = 
        passwordRow.AttemptedPassword 
        |> Array.filter (fun c -> c = passwordRow.TargetCharacter)
        |> Array.length
    targetCharCount >= passwordRow.LowerLimit && targetCharCount <= passwordRow.UpperLimit

let validRows = 
    rows
    |> Seq.filter validPassword
    |> Seq.length

validRows

418

### Part 2

While it appears you validated the passwords correctly, they don't seem to be what the Official Toboggan Corporate Authentication System is expecting.

The shopkeeper suddenly realizes that he just accidentally explained the password policy rules from his old job at the sled rental place down the street! The Official Toboggan Corporate Policy actually works a little differently.

Each policy actually describes two positions in the password, where 1 means the first character, 2 means the second character, and so on. (Be careful; Toboggan Corporate Policies have no concept of "index zero"!) Exactly one of these positions must contain the given letter. Other occurrences of the letter are irrelevant for the purposes of policy enforcement.

Given the same example list from above:

1-3 a: abcde is valid: position 1 contains a and position 3 does not.
1-3 b: cdefg is invalid: neither position 1 nor position 3 contains b.
2-9 c: ccccccccc is invalid: both position 2 and position 9 contain c.
How many passwords are valid according to the new interpretation of the policies?

In [1]:
#!fsharp
let validTobogganCompanyPassword passwordRow = 
    let position1Valid = passwordRow.AttemptedPassword.[passwordRow.LowerLimit - 1] = passwordRow.TargetCharacter
    let position2Valid = passwordRow.AttemptedPassword.[passwordRow.UpperLimit - 1] = passwordRow.TargetCharacter
    position1Valid && not position2Valid || position2Valid && not position1Valid

let validRows = 
    rows 
    |> Seq.filter validTobogganCompanyPassword
    |> Seq.length

validRows

616

## Day 3

With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's very minimal steering and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.

Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (.) and trees (#) you can see. For example:

..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:

..##.........##.........##.........##.........##.........##.......  --->
#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.##.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........#.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...##....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
You start on the open square (.) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).

The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by counting all the trees you would encounter for the slope right 3, down 1:

From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.

The locations you'd check in the above example are marked here with O where there was an open square and X where there was a tree:

..##.........##.........##.........##.........##.........##.......  --->
#..O#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....X..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#O#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..X...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.X#.......#.##.......#.##.......#.##.......#.##.....  --->
.#.#.#....#.#.#.#.O..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........X.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.X#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#X....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...X.#.#..#...#.#.#..#...#.#.#..#...#.#  --->
In this example, traversing the map using this slope would cause you to encounter 7 trees.

Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

### Input

In [1]:
#!fsharp
let hill = File.ReadAllLines("./Day3Input.txt")

### Part 1

In [1]:
#!fsharp
let mutable treeCount = 0
let mutable currentRow = 0
let mutable currentColumn = 0

while currentRow < hill.Length do
    let columnToCheck =
        if currentColumn >= hill.[currentRow].Length then currentColumn - hill.[currentRow].Length
        else currentColumn
    if hill.[currentRow].[columnToCheck] = '#' then
        treeCount <- treeCount + 1
    
    currentRow <- currentRow + 1
    currentColumn <- columnToCheck + 3

treeCount

### Part 2

Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.

Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:

Right 1, down 1.
Right 3, down 1. (This is the slope you already checked.)
Right 5, down 1.
Right 7, down 1.
Right 1, down 2.
In the above example, these slopes would find 2, 7, 3, 4, and 2 tree(s) respectively; multiplied together, these produce the answer 336.

What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [1]:
#!fsharp
type TraversalPattern =
    {
        Horizontal : int
        Vertical : int
    }

let checkSlope pattern =
    let mutable treeCount = 0
    let mutable currentRow = 0
    let mutable currentColumn = 0

    while currentRow < hill.Length do
        let columnToCheck =
            if currentColumn >= hill.[currentRow].Length then currentColumn - hill.[currentRow].Length
            else currentColumn
        if hill.[currentRow].[columnToCheck] = '#' then
            treeCount <- treeCount + 1
        
        currentRow <- currentRow + pattern.Vertical
        currentColumn <- columnToCheck + pattern.Horizontal

    treeCount

let patterns =
    [
        { Horizontal = 1; Vertical = 1 }
        { Horizontal = 3; Vertical = 1 }
        { Horizontal = 5; Vertical = 1 }
        { Horizontal = 7; Vertical = 1 }
        { Horizontal = 1; Vertical = 2 }
    ]

patterns 
|> List.map checkSlope 
|> List.reduce (*)

2138320800